In [ ]:
%pip install -q ipython-autotime tenacity selenium
%load_ext autotime

In [1]:
import selenium.webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tenacity import retry, stop_after_attempt, wait_fixed
import concurrent.futures
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import json
from pprint import pprint
import pandas as pd
import time
from queue import Queue
import threading

In [2]:
def get_chrome_options():
  chrome_options = Options()
  chrome_options.add_argument("--headless=new")
  chrome_options.add_argument("--disable-gpu")
  chrome_options.add_argument("--no-sandbox")
  chrome_options.add_argument("--disable-dev-shm-usage")
  chrome_options.add_argument("--disable-extensions")
  chrome_options.add_argument("--disable-logging")
  chrome_options.add_argument("--log-level=3")
  chrome_options.add_argument("--disable-images")
  chrome_options.add_argument("--disable-notifications")
  chrome_options.add_argument("--disable-web-security")
  chrome_options.page_load_strategy = "eager"
  return chrome_options

drive_path = "C:\\ChromeDriver\\chromedriver.exe"
def get_service():
  service = Service(executable_path=drive_path)
  return service

In [3]:
def get_article_urls(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('h3', class_='title-news') + soup.find_all('h2', class_='title-news')
    urls = [a.find('a')['href'] for a in articles if a.find('a')]
    print(f"URL page: {page_url}, số url: {len(urls)}")
    return urls

In [4]:
# Lấy dữ liệu bài viết từ URL
@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
def get_article_data(url, driver):
      try:
          driver.get(url)
          WebDriverWait(driver, 5).until(
              EC.presence_of_element_located((By.TAG_NAME, "body"))
          )
          print(f"Loaded {url}")

          soup = BeautifulSoup(driver.page_source, 'html.parser')
          data = {}

          data['title'] = soup.find('h1', class_='title-detail').get_text(strip=True) if soup.find('h1', class_='title-detail') else None
          data['description'] = soup.find('p', class_='description').get_text(strip=True) if soup.find('p', class_='description') else None
          data['date'] = soup.find('span', class_='date').get_text(strip=True) if soup.find('span', class_='date') else None
          data['category'] = soup.find('meta', itemprop='articleSection')['content'] if soup.find('meta', itemprop='articleSection') else 'Công nghệ'
          data['content'] = "\n".join([p.get_text(strip=True) for p in soup.find_all('p', class_='Normal')]) or None
          img_tag = soup.find('img', class_='lazy')
          data['thumbnail'] = 'https:' + (img_tag.get('data-src') or img_tag.get('src')) if img_tag and not (img_tag.get('data-src') or img_tag.get('src')).startswith('http') else img_tag.get('data-src') or img_tag.get('src') if img_tag else None
          author_tag = soup.find('span', class_='author_mail')
          data['author'] = author_tag.get_text(strip=True) if author_tag else None
          if not data['author']:
            authors = soup.find_all('p', class_='Normal', style='text-align:right;')
            # if !authors:
            if not authors:
                authors = soup.find_all('p', class_='Normal', style='align: right;')
            if not authors:
                authors = soup.find_all('p', class_='Normal')[-1].get_text(strip=True)
            if authors:
                data['author'] = authors
            else:
                data['author'] = "Không xác định"
          data['tags'] = soup.find('meta', attrs={'name': 'its_tag'})['content'].split(', ') if soup.find('meta', attrs={'name': 'its_tag'}) else []
          data['url'] = url
          data['group'] = 'Khoa học'
          total_comment_label = soup.find('label', id='total_comment')
        #   print(f"total_comment_label: {total_comment_label}")
          data['nums_of_comments'] = int(total_comment_label.get_text(strip=True)) if total_comment_label else 0

          return data
      except Exception as e:
          print(f"Error processing {url}: {e}")
          raise

In [5]:
# url = "https://vnexpress.net/facebook-them-lua-chon-quay-ve-nguyen-ban-4866947.html"
url = "https://vnexpress.net/ten-lua-cua-cong-ty-chau-au-phat-no-khi-chua-toi-quy-dao-4867830.html"
service = Service(executable_path=drive_path)
chrome_options = get_chrome_options()
driver = selenium.webdriver.Chrome(service=service, options=chrome_options)
article_info = get_article_data(url, driver)
pprint(article_info)
driver.quit()

Loaded https://vnexpress.net/ten-lua-cua-cong-ty-chau-au-phat-no-khi-chua-toi-quy-dao-4867830.html
{'author': 'An Khang(TheoSpace)',
 'category': 'Vũ trụ',
 'content': 'Công ty Isar Aerospace tìm cách phóng tên lửa đầu tiên lên quỹ '
            'đạo từ châu Âu vào sáng ngày 20/3. Tên lửa Spectrum của công ty '
            'cất cánh từ cảng vũ trụ Andøya của châu Âu tại Na Uy, nhưng gặp '
            'trục trặc 18 giây sau khi phóng, theoSpace. Video ghi hình buổi '
            'phóng cho thấy tên lửa rơi xuống sau khi bay vài giây, lao xuống '
            'lớp băng bên dưới và phát nổ thành cầu lửa sáng rực.\n'
            'Các kỹ sư chưa rõ nguyên nhân gây ra trục trặc. Từ video về tai '
            'nạn, tên lửa rơi xuống đất với động cơ đã tắt, vì vậy nhiều khả '
            'năng nhân viên điều khiển bay kết thúc chuyến bay ngay khi phương '
            'tiện bắt đầu rơi. Tên lửa Spectrum không chở thiết bị nào trong '
            'chuyến bay thử nghiệm đầu tiên. Isar Aerospace ch

In [6]:
def fetch_all_articles(unique_urls, max_workers=5):
    queue = Queue()
    for url in unique_urls:
        queue.put(url)

    results = []
    failed_urls = []

    def worker():
        with selenium.webdriver.Chrome(service=get_service(), options=get_chrome_options()) as browser:
            while not queue.empty():
                try:
                    url = queue.get_nowait()
                except Queue.Empty:
                    break
                try:
                    article_info = get_article_data(url, browser)
                    if article_info:
                        results.append(article_info)
                    else:
                        failed_urls.append(url)
                except Exception as e:
                    print(f"Failed to process {url}: {e}")
                    failed_urls.append(url)
                finally:
                    queue.task_done()

    threads = []
    for _ in range(max_workers):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

    # Retry failed URLs sequentially
    if failed_urls:
        print(f"Retrying {len(failed_urls)} failed URLs...")
        with selenium.webdriver.Chrome(service=get_service(), options=get_chrome_options()) as browser:
            for url in failed_urls[:]:  # Copy list to modify during iteration
                try:
                    article_info = get_article_data(url, driver)
                    if article_info:
                        results.append(article_info)
                        failed_urls.remove(url)
                except Exception as e:
                    print(f"Retry failed for {url}: {e}")

    print(f"Đã thu thập {len(results)} bài báo, thất bại {len(failed_urls)} URL")
    return results, failed_urls

In [7]:
# base_url = "https://vnexpress.net/cong-nghe"
base_url = "https://vnexpress.net/khoa-hoc"
all_urls_page = []
for page in range(1, 21):
    page_url = f"{base_url}-p{page}"
    article_urls = get_article_urls(page_url)
    all_urls_page.extend(article_urls)

unique_urls = list(set(all_urls_page))
print(f"Tổng số URL duy nhất: {len(unique_urls)}")

URL page: https://vnexpress.net/khoa-hoc-p1, số url: 38
URL page: https://vnexpress.net/khoa-hoc-p2, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p3, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p4, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p5, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p6, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p7, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p8, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p9, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p10, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p11, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p12, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p13, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p14, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p15, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p16, số url: 29
URL page: https://vnexpress.net/khoa-hoc-p17, số url: 29
URL page: https://vnexpress.net/khoa-hoc

In [ ]:
all_data, failed_urls = fetch_all_articles(unique_urls, max_workers=8)
print(f"Số bài báo thu thập được: {len(all_data)}")
print(f"Số URL thất bại: {len(failed_urls)}")
from pprint import pprint
pprint(all_data[:5])

Loaded https://vnexpress.net/nhung-thanh-pho-thong-minh-su-dung-ai-de-giam-tac-nghen-4865431.html
Loaded https://vnexpress.net/nhung-nguyen-to-nguy-hiem-nhat-trong-bang-tuan-hoan-4864031.html
Loaded https://vnexpress.net/viet-nam-anh-huong-rat-nho-tu-dong-dat-o-myanmar-4867089.html
Loaded https://vnexpress.net/lo-nhiet-hach-lon-nhat-the-gioi-su-dung-ai-de-tang-hieu-qua-4866829.html
Loaded https://vnexpress.net/robot-co-the-tu-dung-day-tu-moi-tu-the-nga-4857013.html
Loaded https://vnexpress.net/ly-do-con-nguoi-kho-thay-doi-thoi-quen-xau-4864898.html
Loaded https://vnexpress.net/he-thong-phong-ten-lua-bang-cong-nghe-dem-tu-4866403.html
Loaded https://vnexpress.net/nhan-dien-van-de-then-chot-cua-nen-kinh-te-de-gdp-but-pha-4851854.html
Loaded https://vnexpress.net/phan-lap-thanh-cong-hop-chat-ho-tro-dieu-tri-ung-thu-tu-cay-rieng-4845691.html
Loaded https://vnexpress.net/de-xuat-uu-tien-phat-trien-cong-nghe-chien-luoc-linh-vuc-hat-nhan-4864349.html
Loaded https://vnexpress.net/nha-khoa-hoc-

In [ ]:
print(f"Số bài báo thu thập được: {len(all_data)}") 
print(f"Số URL thất bại: {len(failed_urls)}")
from pprint import pprint
pprint(all_data[0])

In [ ]:
all_data_2, failed_urls_2 = fetch_all_articles(failed_urls, max_workers=5)
print(f"Số bài báo thu thập được: {len(all_data_2)}")
print(f"Số URL thất bại: {len(failed_urls_2)}")
from pprint import pprint
pprint(all_data_2[0])

In [ ]:
all_data.extend(all_data_2)
print(f"Số bài báo thu thập được: {len(all_data)}")

In [ ]:
import pandas as pd
import json

# Tạo một list để lưu trữ các dòng dữ liệu
rows = []

# Lặp qua mỗi bài viết
for article in all_data:
  if article is not None and isinstance(article, dict):
    title = article['title'] if article['title'] is not None else ''
    description = article['description'] if article['description'] is not None else ''
    date = article['date'] if article['date'] is not None else ''
    category = article['category'] if article['category'] is not None else ''
    thumbnail = article['thumbnail'] if article['thumbnail'] is not None else ''
    content = article['content'] if article['content'] is not None else ''
    author = article['author'] if article['author'] is not None else ''
    tags = ', '.join(article['tags'])  if article['tags'] is not None else ''
    group = article['group'] if article['group'] is not None else ''
    nums_of_comments = article['nums_of_comments'] if article['nums_of_comments'] is not None else 0
    url = article['url'] if article['url'] is not None else ''

    # Thêm dòng dữ liệu vào list
    rows.append({
        'title': title,
        'description': description,
        'date': date,
        'category': category,
        'thumbnail': thumbnail,
        'content': content,
        'author': author,
        'tags': tags,
        'group': group,
        'nums_of_comments': nums_of_comments,
        'url': url,
    })
  else:
    print(f"Skipping invalid article: {article}")

# Tạo DataFrame từ list các dòng dữ liệu
df = pd.DataFrame(rows)

# Lưu DataFrame thành file CSV
df.to_csv('vnexpress_congnghe_raw_data.csv', index=False, encoding='utf-8-sig')

print("DataFrame đã được lưu thành file all_data.csv")